# 추천 시스템 (Recommender Systems)

* 추천 시스템은 크게 두가지로 구분 가능
  * 컨텐츠 기반 필터링 (content-based filtering)
  * 협업 필터링 (collaborative filtering)
* 두가지를 조합한 hybrid 방식도 가능
* 컨텐츠 기반 필터링은 지금까지 사용자의 이전 행동과 명시적 피드백을 통해 사용자가 좋아하는 것과 유사한 항목을 추천
* 협업 필터링은 사용자와 항목간의 유사성을 동시에 사용해 추천

## Surprise

* 추천 시스템 개발을 위한 라이브러리
* 다양한 모델과 데이터 제공
* scikit-learn과 유사한 사용 방법

In [1]:
!pip install surprise

     |████████████████████████████████| 11.8 MB 4.1 MB/s eta 0:00:01
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=2026230 sha256=8441e67e4dc7ef724ad8400bad9e7b49279b46e9ea9b0416975a4466b4817343
  Stored in directory: /home/kangjunekoo/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [2]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate

간단한 surprise 실습

In [4]:
data=Dataset.load_builtin('ml-100k',prompt=False)
data.raw_ratings[:10]

Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /home/kangjunekoo/.surprise_data/ml-100k


[('196', '242', 3.0, '881250949'),
 ('186', '302', 3.0, '891717742'),
 ('22', '377', 1.0, '878887116'),
 ('244', '51', 2.0, '880606923'),
 ('166', '346', 1.0, '886397596'),
 ('298', '474', 4.0, '884182806'),
 ('115', '265', 2.0, '881171488'),
 ('253', '465', 5.0, '891628467'),
 ('305', '451', 3.0, '886324817'),
 ('6', '86', 3.0, '883603013')]

In [5]:
model=SVD()

In [6]:
cross_validate(model,data,measures=['rmse','mae'],cv=5,verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9385  0.9303  0.9411  0.9380  0.9337  0.9363  0.0038  
MAE (testset)     0.7392  0.7324  0.7421  0.7382  0.7376  0.7379  0.0032  
Fit time          2.72    2.68    2.68    2.69    2.71    2.70    0.02    
Test time         0.14    0.08    0.14    0.09    0.13    0.12    0.03    


{'test_rmse': array([0.93850522, 0.93030108, 0.94105412, 0.93795639, 0.93367353]),
 'test_mae': array([0.73924136, 0.73240973, 0.74209833, 0.7382465 , 0.7375728 ]),
 'fit_time': (2.7170369625091553,
  2.681189775466919,
  2.6766645908355713,
  2.6929848194122314,
  2.707291841506958),
 'test_time': (0.14100241661071777,
  0.08498072624206543,
  0.13762998580932617,
  0.08507990837097168,
  0.13309240341186523)}

## 컨텐츠 기반 필터링 (Content-based Filtering)

* 컨텐츠 기반 필터링은 이전의 행동과 명시적 피드백을 통해 좋아하는 것과 유사한 항목을 추천
  * ex) 내가 지금 까지 시청한 영화 목록과 다른 사용자의 시청 목록을 비교해 나와 비슷한 취향의 사용자가 시청한 영화를 추천
* 유사도를 기반으로 추천
* 컨텐츠 기반 필터링은 다음과 같은 장단점이 있다.
  * 장점
    * 많은 수의 사용자를 대상으로 쉽게 확장 가능
    * 사용자가 관심을 갖지 않던 상품 추천 가능
  * 단점
    * 입력 특성을 직접 설계해야 하기 때문에 많은 도메인 지식이 필요
    * 사용자의 기존 관심사항을 기반으로만 추천 가능

In [7]:
import numpy as np
from surprise import Dataset

* 이진 벡터의 내적을 통해 다른 사용자들과의 유사도 구하기
* 나와 가장 높은 유사도를 가진 사용자의 시청 목록을 추천

In [8]:
data=Dataset.load_builtin('ml-100k',prompt=False)
raw_data=np.array(data.raw_ratings,dtype=int)

In [9]:
raw_data[:,0]-=1
raw_data[:,1]-=1

In [10]:
n_users=np.max(raw_data[:,0])
n_movies=np.max(raw_data[:,1])
shape=(n_users+1,n_movies+1)
shape

(943, 1682)

In [52]:
import pandas as pd
df_raw_data=pd.DataFrame(raw_data)
df_raw_data

,0,1,2,3
0,195,241,3,881250949
1,185,301,3,891717742
2,21,376,1,878887116
3,243,50,2,880606923
4,165,345,1,886397596
...,...,...,...,...
99995,879,475,3,880175444
99996,715,203,5,879795543
99997,275,1089,1,874795795
99998,12,224,2,882399156


In [55]:
raw_data

array([[      195,       241,         3, 881250949],
       [      185,       301,         3, 891717742],
       [       21,       376,         1, 878887116],
       ...,
       [      275,      1089,         1, 874795795],
       [       12,       224,         2, 882399156],
       [       11,       202,         3, 879959583]])

In [57]:
user_id,movie_id,rating,time = raw_data

ValueError: too many values to unpack (expected 4)

In [11]:
adj_matrix=np.ndarray(shape,dtype=int)
for user_id,movie_id,rating,time in raw_data:
  adj_matrix[user_id][movie_id]=1.
adj_matrix

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]])

In [12]:
my_id,my_vector=0,adj_matrix[0]
best_match,best_match_id,best_match_vector=-1,-1,[]

for user_id,user_vector in enumerate(adj_matrix):
  if my_id !=user_id:
    similarity=np.dot(my_vector,user_vector)
    if similarity > best_match:
      best_match=similarity
      best_match_id=user_id
      best_match_vector=user_vector

  
print('best Match: {}, Best Match ID : {}'.format(best_match,best_match_id))

best Match: 183, Best Match ID : 275


In [13]:
recommend_list=[]
for i, log in enumerate(zip(my_vector,best_match_vector)):
  log1,log2=log
  if log1<1. and log2 > 0.:
    recommend_list.append(i)
print(recommend_list)

[272, 273, 275, 280, 281, 283, 287, 288, 289, 290, 292, 293, 297, 299, 300, 301, 302, 306, 312, 314, 315, 316, 317, 321, 322, 323, 324, 327, 330, 331, 332, 333, 339, 342, 345, 346, 353, 354, 355, 356, 357, 363, 364, 365, 366, 372, 374, 378, 379, 381, 382, 383, 384, 385, 386, 387, 390, 391, 392, 394, 395, 396, 398, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 412, 414, 416, 417, 418, 419, 420, 422, 424, 425, 426, 427, 428, 430, 431, 432, 435, 442, 446, 447, 448, 449, 450, 451, 452, 454, 455, 457, 460, 461, 462, 468, 469, 470, 471, 472, 473, 474, 478, 495, 500, 507, 517, 522, 525, 530, 539, 540, 543, 545, 546, 548, 549, 550, 551, 553, 557, 558, 560, 561, 562, 563, 565, 566, 567, 568, 570, 571, 574, 575, 576, 577, 580, 581, 582, 585, 587, 589, 590, 594, 596, 602, 623, 626, 627, 630, 633, 635, 639, 646, 648, 651, 652, 654, 657, 664, 668, 671, 677, 678, 681, 683, 684, 685, 690, 691, 692, 695, 696, 708, 709, 714, 718, 719, 720, 724, 726, 727, 731, 733, 734, 736, 738, 741, 742, 745,

* 유클리드 거리를 사용해 추천
$$euclidean = \sqrt{\sum_{d=1}^{D}(A_i - B_i)^2}$$
* 거리가 가까울 수록(값이 작을 수록) 나와 유사한 사용자

In [14]:
my_id,my_vector=0,adj_matrix[0]
best_match,best_match_id,best_match_vector=9999,-1,[]

for user_id,user_vector in enumerate(adj_matrix):
  if my_id !=user_id:
    euclidean_dist=np.sqrt(np.sum(np.square(my_vector-user_vector)))
    if euclidean_dist < best_match:
      best_match=euclidean_dist
      best_match_id=user_id
      best_match_vector=user_vector

  
print('best Match: {}, Best Match ID : {}'.format(best_match,best_match_id))

best Match: 14.832396974191326, Best Match ID : 737


In [15]:
recommend_list=[]
for i, log in enumerate(zip(my_vector,best_match_vector)):
  log1,log2=log
  if log1<1. and log2 > 0.:
    recommend_list.append(i)
print(recommend_list)

[297, 312, 317, 342, 356, 366, 379, 384, 392, 402, 404, 407, 417, 422, 428, 433, 448, 454, 469, 473, 495, 510, 516, 526, 527, 549, 567, 602, 635, 649, 650, 654, 658, 661, 664, 696, 731, 746, 750, 754, 915, 918, 925, 929, 950, 968, 1015, 1046]


* 코사인 유사도를 사용해 추천

\begin{equation}
cos \theta = \frac{A \cdot B}{||A|| \times ||B||}
\end{equation}
* 두 벡터가 이루고 있는 각을 계산

In [16]:
def compute_cos_similarity(v1,v2):
  norm1=np.sqrt(np.sum(np.square(v1)))
  norm2=np.sqrt(np.sum(np.square(v2)))
  dot=np.dot(v1,v2)
  return dot/(norm1*norm2)

In [17]:
my_id,my_vector=0,adj_matrix[0]
best_match,best_match_id,best_match_vector=-1,-1,[]

for user_id,user_vector in enumerate(adj_matrix):
  if my_id !=user_id:
    cos_similarity=compute_cos_similarity(my_vector,user_vector)
    if cos_similarity > best_match:
      best_match=cos_similarity
      best_match_id=user_id
      best_match_vector=user_vector

  
print('best Match: {}, Best Match ID : {}'.format(best_match,best_match_id))

best Match: 0.5278586163659506, Best Match ID : 915


915 qjsdlqhkTwlaks sksms dksqhs dudghkemfdmf cncjsgownsek.

In [18]:
recommend_list=[]
for i, log in enumerate(zip(my_vector,best_match_vector)):
  log1,log2=log
  if log1<1. and log2 > 0.:
    recommend_list.append(i)
print(recommend_list)

[272, 275, 279, 280, 283, 285, 289, 294, 297, 316, 317, 355, 365, 366, 368, 379, 380, 381, 384, 386, 392, 398, 401, 404, 416, 420, 422, 424, 426, 427, 430, 432, 450, 460, 461, 466, 469, 471, 473, 474, 475, 479, 482, 483, 497, 505, 508, 510, 511, 522, 526, 527, 529, 530, 534, 536, 540, 545, 548, 549, 556, 557, 558, 560, 565, 567, 568, 569, 577, 580, 581, 582, 592, 596, 630, 635, 639, 641, 649, 651, 654, 673, 677, 678, 683, 684, 692, 696, 701, 703, 707, 708, 709, 712, 714, 719, 720, 726, 731, 734, 736, 738, 740, 745, 747, 754, 755, 761, 762, 763, 766, 780, 789, 791, 805, 819, 823, 824, 830, 843, 862, 865, 918, 929, 930, 938, 942, 943, 947, 958, 959, 960, 970, 977, 1004, 1008, 1009, 1010, 1013, 1041, 1045, 1069, 1072, 1073, 1078, 1097, 1100, 1108, 1112, 1118, 1134, 1193, 1205, 1207, 1216, 1219, 1267, 1334, 1400, 1427, 1596, 1681]


기존 방법에 명시적 피드백(사용자가 평가한 영화 점수)을 추가해 실험

In [19]:
adj_matrix=np.ndarray(shape,dtype=int)
for user_id,movie_id,rating,time in raw_data:
  adj_matrix[user_id][movie_id]=rating
adj_matrix

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]])

In [20]:
my_id,my_vector=0,adj_matrix[0]
best_match,best_match_id,best_match_vector=9999,-1,[]

for user_id,user_vector in enumerate(adj_matrix):
  if my_id !=user_id:
    euclidean_dist=np.sqrt(np.sum(np.square(my_vector-user_vector)))
    if euclidean_dist < best_match:
      best_match=euclidean_dist
      best_match_id=user_id
      best_match_vector=user_vector

  
print('best Match: {}, Best Match ID : {}'.format(best_match,best_match_id))

best Match: 55.06359959174482, Best Match ID : 737


In [21]:
my_id,my_vector=0,adj_matrix[0]
best_match,best_match_id,best_match_vector=-1,-1,[]

for user_id,user_vector in enumerate(adj_matrix):
  if my_id !=user_id:
    cos_similarity=compute_cos_similarity(my_vector,user_vector)
    if cos_similarity > best_match:
      best_match=cos_similarity
      best_match_id=user_id
      best_match_vector=user_vector

  
print('best Match: {}, Best Match ID : {}'.format(best_match,best_match_id))

best Match: 0.569065731527988, Best Match ID : 915


## 협업 필터링(Collaborative Filtering)

* 사용자와 항목의 유사성을 동시에 고려해 추천
* 기존에 내 관심사가 아닌 항목이라도 추천 가능
* 자동으로 임베딩 학습 가능
* 협업 필터링은 다음과 같은 장단점을 갖고 있다.
  * 장점
    * 자동으로 임베딩을 학습하기 때문에 도메인 지식이 필요 없다.
    * 기존의 관심사가 아니더라도 추천 가능
  * 단점
    * 학습 과정에 나오지 않은 항목은 임베딩을 만들 수 없음
    * 추가 특성을 사용하기 어려움

In [22]:
from surprise import KNNBasic, SVD, SVDpp,NMF
from surprise import Dataset#
from surprise.model_selection import cross_validate

In [23]:
data=Dataset.load_builtin('ml-100k',prompt=False)

* KNN을 사용한 협업 필터링

In [24]:
model=KNNBasic()
cross_validate(model,data,measures=['rmse','mae'],cv=5,n_jobs=4,verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9822  0.9897  0.9691  0.9783  0.9720  0.9783  0.0073  
MAE (testset)     0.7741  0.7818  0.7671  0.7725  0.7680  0.7727  0.0053  
Fit time          0.13    0.13    0.14    0.14    0.16    0.14    0.01    
Test time         1.77    1.84    1.83    1.77    1.72    1.79    0.04    


{'test_rmse': array([0.98222211, 0.98972405, 0.96913931, 0.97833766, 0.97201354]),
 'test_mae': array([0.77413565, 0.78180296, 0.76708171, 0.77247445, 0.76800712]),
 'fit_time': (0.13117051124572754,
  0.13125371932983398,
  0.13594269752502441,
  0.13716983795166016,
  0.157395601272583),
 'test_time': (1.770310878753662,
  1.840428352355957,
  1.832695484161377,
  1.7656188011169434,
  1.7222816944122314)}

* SVD를 사용한 협업 필터링

In [25]:
model=SVD()
cross_validate(model,data,measures=['rmse','mae'],cv=5,n_jobs=4,verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9412  0.9326  0.9311  0.9412  0.9340  0.9360  0.0043  
MAE (testset)     0.7410  0.7339  0.7342  0.7414  0.7384  0.7378  0.0032  
Fit time          2.52    2.51    2.54    2.54    2.76    2.57    0.09    
Test time         0.11    0.11    0.10    0.11    0.10    0.11    0.00    


{'test_rmse': array([0.9412454 , 0.93262269, 0.93106339, 0.94122044, 0.93404252]),
 'test_mae': array([0.74099816, 0.73392108, 0.73419173, 0.74138633, 0.7383776 ]),
 'fit_time': (2.51523494720459,
  2.5069546699523926,
  2.5386664867401123,
  2.5374951362609863,
  2.757296323776245),
 'test_time': (0.10702681541442871,
  0.10738635063171387,
  0.10348081588745117,
  0.10901355743408203,
  0.10022354125976562)}

* NMF를 사용한 협업 필터링

In [26]:
model=NMF()
cross_validate(model,data,measures=['rmse','mae'],cv=5,n_jobs=4,verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9576  0.9556  0.9667  0.9643  0.9605  0.9610  0.0041  
MAE (testset)     0.7510  0.7527  0.7609  0.7593  0.7541  0.7556  0.0038  
Fit time          2.43    2.49    2.51    2.45    2.45    2.47    0.03    
Test time         0.09    0.09    0.09    0.09    0.09    0.09    0.00    


{'test_rmse': array([0.95764165, 0.95560365, 0.96670845, 0.96434177, 0.96051125]),
 'test_mae': array([0.75098091, 0.75272954, 0.76094644, 0.75925439, 0.75405927]),
 'fit_time': (2.4335782527923584,
  2.4916529655456543,
  2.511504650115967,
  2.450873613357544,
  2.4474399089813232),
 'test_time': (0.09008288383483887,
  0.09021210670471191,
  0.08599328994750977,
  0.08683514595031738,
  0.08668899536132812)}

* SVD++를 사용한 협업 필터링

In [27]:
model=SVDpp()
cross_validate(model,data,measures=['rmse','mae'],cv=5,n_jobs=4,verbose=True)

Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9178  0.9266  0.9218  0.9153  0.9215  0.9206  0.0039  
MAE (testset)     0.7182  0.7289  0.7214  0.7191  0.7246  0.7225  0.0039  
Fit time          103.56  103.05  102.79  102.12  102.10  102.72  0.56    
Test time         1.92    1.82    1.81    1.82    1.81    1.84    0.04    


{'test_rmse': array([0.91775758, 0.92663867, 0.92175379, 0.91530555, 0.92152871]),
 'test_mae': array([0.71823459, 0.72894813, 0.72140623, 0.71911526, 0.7246224 ]),
 'fit_time': (103.55862498283386,
  103.05499076843262,
  102.78749251365662,
  102.12325286865234,
  102.09960842132568),
 'test_time': (1.9170722961425781,
  1.8241431713104248,
  1.8094685077667236,
  1.8181326389312744,
  1.8115644454956055)}

## 하이브리드(Hybrid)

* 컨텐츠 기반 필터링과 협업 필터링을 조합한 방식
* 많은 하이브리드 방식이 존재
* 실습에서는 협업 필터링으로 임베딩을 학습하고 컨텐츠 기반 필터링으로 유사도 기반 추천을 수행하는 추천 엔진 개발

In [28]:
import numpy as np
from sklearn.decomposition import randomized_svd,non_negative_factorization
from surprise import Dataset

In [29]:
data=Dataset.load_builtin('ml-100k',prompt=False)
raw_data=np.array(data.raw_ratings,dtype=int)
raw_data[:,0]-=1
raw_data[:,1]-=1

In [30]:
n_users=np.max(raw_data[:,0])
n_movies=np.max(raw_data[:,1])
shape=(n_users+1,n_movies+1)
shape

(943, 1682)

In [31]:
n_users

942

In [70]:
adj_matrix=np.ndarray(shape,dtype=int)
for user_id,movie_id,rating,time in raw_data:
  adj_matrix[user_id][movie_id]=rating
adj_matrix

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]])

In [33]:
import pandas as pd

In [34]:
df_adj_matrix=pd.DataFrame(adj_matrix)
df_adj_matrix

,0,1,2,3,4,5,6,7,8,9,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
0,5,3,4,3,3,5,4,1,5,3,...,0,0,0,0,0,0,0,0,0,0
1,4,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,0,0,0,0,0,0,0,0,5,0,...,0,0,0,0,0,0,0,0,0,0
939,0,0,0,2,0,0,4,5,3,0,...,0,0,0,0,0,0,0,0,0,0
940,5,0,0,0,0,0,4,0,0,0,...,0,0,0,0,0,0,0,0,0,0
941,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


U =user, V= item ,S= tri value vector

In [36]:
U,S,V=randomized_svd(adj_matrix,n_components=2)
S=np.diag(S)

/home/kangjunekoo/anaconda3/envs/pytorch/lib/python3.7/site-packages/sklearn/utils/extmath.py:376: FutureWarning: If 'random_state' is not supplied, the current default is to use 0 as a fixed seed. This will change to  None in version 1.2 leading to non-deterministic results that better reflect nature of the randomized_svd solver. If you want to silence this warning, set 'random_state' to an integer seed or to None explicitly depending if you want your code to be deterministic or not.
  FutureWarning,


In [39]:
df_U=pd.DataFrame(U)
df_U

,0,1
0,0.065804,0.005975
1,0.014021,-0.046626
2,0.005658,-0.025618
3,0.005993,-0.020698
4,0.032747,0.009159
...,...,...
938,0.011282,-0.041178
939,0.030006,-0.008428
940,0.007445,-0.025021
941,0.024031,0.008096


In [41]:
df_S=pd.DataFrame(S)
df_S

,0,1
0,640.633623,0.000000
1,0.000000,244.836346


In [40]:
df_V=pd.DataFrame(V)
df_V

,0,1,2,3,4,5,6,7,8,9,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
0,0.095951,0.035180,0.019929,0.059952,0.021607,0.005367,0.085673,0.063814,0.065295,0.022075,...,0.000150,0.000358,0.000176,0.000117,0.000220,0.000015,0.000046,0.000030,0.000331,0.000317
1,-0.087240,-0.007025,-0.028618,0.013050,-0.015311,0.002251,-0.079269,0.027715,-0.042917,0.007884,...,0.000292,0.001386,-0.000770,-0.000513,-0.000215,-0.000224,-0.000672,-0.000448,0.000105,0.000203


In [72]:
print(U.shape)
print(S.shape)
print(V.shape)

(943, 2)
(2, 2)
(2, 1682)


In [43]:
df_USV=np.matmul(np.matmul(U,S),V)
df_USV=pd.DataFrame(df_USV)
df_USV

,0,1,2,3,4,5,6,7,8,9,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
0,3.917327,1.472766,0.798262,2.546465,0.888481,0.229530,3.495722,2.730697,2.689811,0.942156,...,0.006770,0.017122,0.006273,0.004182,0.008968,0.000312,0.000937,0.000625,0.014110,0.013655
1,1.857772,0.396191,0.505706,0.389534,0.368863,0.022513,1.674457,0.256811,1.076432,0.108290,...,-0.001976,-0.012612,0.010362,0.006908,0.004432,0.002694,0.008083,0.005389,0.001772,0.000527
2,0.894990,0.171578,0.251739,0.135453,0.174352,0.005336,0.807738,0.057469,0.505864,0.030567,...,-0.001283,-0.007399,0.005463,0.003642,0.002146,0.001460,0.004381,0.002921,0.000540,-0.000126
3,0.810501,0.170669,0.221544,0.164044,0.160548,0.009199,0.730645,0.104553,0.468185,0.044803,...,-0.000900,-0.005652,0.004574,0.003049,0.001935,0.001194,0.003581,0.002388,0.000738,0.000187
4,1.817325,0.722279,0.353912,1.287004,0.418963,0.117633,1.619587,1.400895,1.273581,0.480801,...,0.003811,0.010620,0.001956,0.001304,0.004137,-0.000184,-0.000552,-0.000368,0.007181,0.007103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,1.573066,0.325097,0.432570,0.301751,0.310532,0.016099,1.418415,0.181812,0.904629,0.080072,...,-0.001852,-0.011391,0.009028,0.006018,0.003759,0.002369,0.007106,0.004738,0.001332,0.000242
939,2.024478,0.690753,0.442143,1.125537,0.446946,0.098519,1.810464,1.169505,1.343719,0.408090,...,0.002291,0.004022,0.004962,0.003308,0.004676,0.000754,0.002263,0.001509,0.006147,0.005672
940,0.992052,0.210815,0.270363,0.205977,0.196843,0.011807,0.894203,0.134556,0.574321,0.056985,...,-0.001068,-0.006786,0.005552,0.003701,0.002367,0.001445,0.004335,0.002890,0.000934,0.000267
941,1.304254,0.527670,0.250080,0.948844,0.302297,0.087081,1.161830,1.037358,0.920154,0.355483,...,0.002894,0.008260,0.001176,0.000784,0.002964,-0.000210,-0.000631,-0.000421,0.005305,0.005281


In [45]:
df_U=pd.DataFrame(U)
df_U

,0,1
0,0.065804,0.005975
1,0.014021,-0.046626
2,0.005658,-0.025618
3,0.005993,-0.020698
4,0.032747,0.009159
...,...,...
938,0.011282,-0.041178
939,0.030006,-0.008428
940,0.007445,-0.025021
941,0.024031,0.008096


In [46]:
U[0]

array([0.06580431, 0.00597507])

* 사용자 기반 추천
* 나와 비슷한 취향을 가진 다른 사용자의 행동을 추천
* 사용자 특징 벡터의 유사도 사용

In [44]:
my_id,my_vector=0,U[0]
best_match,best_match_id,best_match_vector=-1,-1,[]

for user_id,user_vector in enumerate(U):
  if my_id !=user_id:
    cos_similarity=compute_cos_similarity(my_vector,user_vector)
    if cos_similarity > best_match:
      best_match=cos_similarity
      best_match_id=user_id
      best_match_vector=user_vector

  
print('best Match: {}, Best Match ID : {}'.format(best_match,best_match_id))

best Match: 0.9999942295956324, Best Match ID : 235


In [48]:
adj_matrix[my_id]

array([5, 3, 4, ..., 0, 0, 0])

In [49]:
adj_matrix[best_match_id]

array([0, 0, 0, ..., 0, 0, 0])

we can append both of the movie that I havn't seen and the movie that is similar

In [75]:
recommend_list=[]
for i, log in enumerate(zip(adj_matrix[my_id],adj_matrix[best_match_id])):
  log1,log2=log
  if log1<1. and log2 > 0.:
    recommend_list.append(i)
print(recommend_list)

[272, 273, 274, 281, 285, 288, 293, 297, 303, 306, 312, 317, 327, 332, 369, 410, 418, 419, 422, 426, 428, 431, 434, 442, 461, 475, 477, 482, 495, 503, 504, 505, 506, 509, 519, 520, 522, 525, 531, 545, 548, 590, 594, 595, 613, 631, 654, 658, 660, 672, 684, 685, 691, 695, 698, 704, 716, 728, 734, 749, 755, 863, 865, 933, 1012, 1038, 1101, 1327, 1400]


In [50]:
log

(0, 1)

In [51]:
i

1681

* 항목 기반 추천
* 내가 본 항목과 비슷한 항목을 추천
* 항목 특징 벡터의 유사도 사용

In [76]:
my_id,my_vector=0,V.T[0]
best_match,best_match_id,best_match_vector=-1,-1,[]

for user_id,user_vector in enumerate(V.T):
  if my_id !=user_id:
    cos_similarity=compute_cos_similarity(my_vector,user_vector)
    if cos_similarity > best_match:
      best_match=cos_similarity
      best_match_id=user_id
      best_match_vector=user_vector

  
print('best Match: {}, Best Match ID : {}'.format(best_match,best_match_id))

best Match: 0.9999999951364144, Best Match ID : 1287


In [77]:
recommend_list=[]
for i, log in enumerate(adj_matrix):
  if adj_matrix[i][my_id]>0.9:
    recommend_list.append(i)
print(recommend_list)

[0, 1, 4, 5, 9, 12, 14, 15, 16, 17, 19, 20, 22, 24, 25, 37, 40, 41, 42, 43, 44, 48, 53, 55, 56, 57, 58, 61, 62, 63, 64, 65, 66, 69, 71, 72, 74, 76, 78, 80, 81, 82, 83, 88, 91, 92, 93, 94, 95, 96, 98, 100, 101, 105, 107, 108, 116, 119, 120, 123, 124, 127, 129, 130, 133, 136, 137, 140, 143, 144, 147, 149, 150, 156, 157, 159, 161, 167, 173, 176, 177, 180, 181, 183, 188, 192, 193, 197, 198, 199, 200, 201, 202, 203, 208, 209, 212, 215, 221, 222, 229, 230, 231, 233, 234, 241, 242, 243, 245, 246, 247, 248, 249, 250, 251, 252, 253, 255, 261, 262, 264, 267, 270, 273, 274, 275, 276, 278, 279, 285, 286, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 300, 302, 304, 306, 307, 310, 311, 312, 313, 319, 321, 323, 324, 325, 326, 329, 330, 331, 335, 337, 338, 339, 342, 343, 344, 346, 347, 349, 356, 358, 359, 362, 364, 370, 373, 377, 378, 379, 380, 386, 387, 388, 389, 392, 393, 394, 395, 397, 398, 400, 401, 402, 405, 406, 410, 411, 415, 416, 418, 421, 423, 424, 428, 431, 433, 434, 437, 440, 444, 

* 비음수 행렬 분해를 사용한 하이브리드 추천

In [78]:
adj_matrix

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]])

In [79]:
A,B,iter=non_negative_factorization(adj_matrix,n_components=2)

/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_nmf.py:294: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  FutureWarning,


In [80]:
np.matmul(A,B)

array([[3.71108858e+00, 1.48454102e+00, 7.39535363e-01, ...,
        3.64490531e-03, 1.45506510e-02, 1.44110916e-02],
       [2.11724416e+00, 2.37338725e-01, 5.51650703e-01, ...,
        4.76092332e-03, 3.03585167e-05, 0.00000000e+00],
       [9.85272360e-01, 1.10447010e-01, 2.56713988e-01, ...,
        2.21552443e-03, 1.41275192e-05, 0.00000000e+00],
       ...,
       [1.04485667e+00, 1.17126289e-01, 2.72238757e-01, ...,
        2.34950819e-03, 1.49818804e-05, 0.00000000e+00],
       [1.45758857e+00, 5.42112787e-01, 2.99181695e-01, ...,
        1.61177659e-03, 5.15921394e-03, 5.10771005e-03],
       [2.44733103e+00, 9.41208708e-01, 4.95740038e-01, ...,
        2.56992125e-03, 9.08287324e-03, 8.99389282e-03]])

* 사용자 기반 추천

In [81]:
my_id,my_vector=0,U[0] #suppose that My ID is the 0
best_match,best_match_id,best_match_vector=-1,-1,[]

for user_id,user_vector in enumerate(U):
  if my_id !=user_id:
    cos_similarity=compute_cos_similarity(my_vector,user_vector)
    if cos_similarity > best_match:
      best_match=cos_similarity
      best_match_id=user_id
      best_match_vector=user_vector

  
print('best Match: {}, Best Match ID : {}'.format(best_match,best_match_id))

best Match: 0.9999942295956324, Best Match ID : 235


In [82]:
recommend_list = []
for i, log in enumerate(zip(adj_matrix[my_id], adj_matrix[best_match_id])):
  log1, log2 = log
  if log1 < 1. and log2 > 0.:
    recommend_list.append(i)
print(recommend_list)

[272, 273, 274, 281, 285, 288, 293, 297, 303, 306, 312, 317, 327, 332, 369, 410, 418, 419, 422, 426, 428, 431, 434, 442, 461, 475, 477, 482, 495, 503, 504, 505, 506, 509, 519, 520, 522, 525, 531, 545, 548, 590, 594, 595, 613, 631, 654, 658, 660, 672, 684, 685, 691, 695, 698, 704, 716, 728, 734, 749, 755, 863, 865, 933, 1012, 1038, 1101, 1327, 1400]


* 항목 기반 추천

In [83]:
my_id,my_vector=0,V.T[0]
best_match,best_match_id,best_match_vector=-1,-1,[]

for user_id,user_vector in enumerate(V.T):
  if my_id !=user_id:
    cos_similarity=compute_cos_similarity(my_vector,user_vector)
    if cos_similarity > best_match:
      best_match=cos_similarity
      best_match_id=user_id
      best_match_vector=user_vector

  
print('best Match: {}, Best Match ID : {}'.format(best_match,best_match_id))

best Match: 0.9999999951364144, Best Match ID : 1287


In [84]:
recommend_list = []
for i, user_vector in enumerate(adj_matrix):
  if adj_matrix[i][my_id] > 0.9:
    recommend_list.append(i)
print(recommend_list)

[0, 1, 4, 5, 9, 12, 14, 15, 16, 17, 19, 20, 22, 24, 25, 37, 40, 41, 42, 43, 44, 48, 53, 55, 56, 57, 58, 61, 62, 63, 64, 65, 66, 69, 71, 72, 74, 76, 78, 80, 81, 82, 83, 88, 91, 92, 93, 94, 95, 96, 98, 100, 101, 105, 107, 108, 116, 119, 120, 123, 124, 127, 129, 130, 133, 136, 137, 140, 143, 144, 147, 149, 150, 156, 157, 159, 161, 167, 173, 176, 177, 180, 181, 183, 188, 192, 193, 197, 198, 199, 200, 201, 202, 203, 208, 209, 212, 215, 221, 222, 229, 230, 231, 233, 234, 241, 242, 243, 245, 246, 247, 248, 249, 250, 251, 252, 253, 255, 261, 262, 264, 267, 270, 273, 274, 275, 276, 278, 279, 285, 286, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 300, 302, 304, 306, 307, 310, 311, 312, 313, 319, 321, 323, 324, 325, 326, 329, 330, 331, 335, 337, 338, 339, 342, 343, 344, 346, 347, 349, 356, 358, 359, 362, 364, 370, 373, 377, 378, 379, 380, 386, 387, 388, 389, 392, 393, 394, 395, 397, 398, 400, 401, 402, 405, 406, 410, 411, 415, 416, 418, 421, 423, 424, 428, 431, 433, 434, 437, 440, 444, 